# [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/enlite-ai/maze/blob/main/tutorials/notebooks/getting_started/getting_started_2_configurability.ipynb) Maze: Getting Started Part II - Configurability

Part 2 of 4 in the *Getting started* series. We recommend reading [part 1](https://colab.research.google.com/github/enlite-ai/maze/blob/master/tutorials/notebooks/getting_started/getting_started_1.ipynb) prior to this notebook.

## On Maze

MazeRL is an application-oriented Deep Reinforcement Learning (RL) framework, addressing real-world decision problems. If this caught your interest, check out
* the [Github repository](https://github.com/enlite-ai/maze),
* the [documentation](https://maze-rl.readthedocs.io/en/latest/index.html#documentation-overview) or
* the official [website](https://www.enlite.ai/).


## Introduction

This notebook discusses configurability in Maze. In particular, it introduces Maze' Hydra-based configuration system and demonstrates how to pass components in different ways to `RunContext` to adapt the agent to your needs.

We highlight the configurability of environments via the OpenAI Gym-inspired wrapper system, which allows wrapping environments in order to modify their behaviour without having to rewrite the original environment.

As in part 1 of the tutorial, we will mostly use the `CartPole-v0` environment (see [here](https://gym.openai.com/envs/CartPole-v0/) for more information).

### Install Maze and Dependencies

Maze is available as pip package. The other dependencies required for this notebook are PyTorch and OpenAI's gym. We recommend installing PyTorch via Conda. If you are executing this notebook on Google Collabe, both libraries are already available.

In [2]:
%%capture
# remove %%capture to see cell output

!pip install torch
!pip install gym
!pip install maze-rl

## The Configuration System

Maze utilizes [Hydra](https://github.com/facebookresearch/hydra) to facilitate a powerful configuration mechanism boosting developers' flexibility in their creation of reinforcement learning projects. Our CLI `maze-run` is built on top of Hydra. When using Maze in Python, the high-level API `RunContext` offers functionality mostly congruent with the CLI and provides additional features.

Our documentation offers more detailed information on [RunContext](https://maze-rl.readthedocs.io/en/latest/concepts_and_structure/run_context_overview.html) and the underlying Hydra-based [configuration system](https://maze-rl.readthedocs.io/en/latest/concepts_and_structure/hydra.html). In this tutorial, we will stick to Python and work only with `RunContext`.

In a nutshell, `RunContext` uses Hydra to build a configuration structure specifying every component to be instantiated. Major components (e.g. policy, environment etc.) are exposed explicitly in `RunContext.__init__` and can be passed in three different ways:
* As instantiated Python object. Environments are a special case here, as they have to be passed as factory functions instead of as instances. In [Getting Started Part I](www.github.com/enlite-ai/maze/blob/main/tutorials/notebooks/getting_started_1.ipynb) we passed our environment using this approach (`...env=lambda: GymMazeEnv(env=gym.make("CartPole-v0")) ...`).
* As configuration module name. This will prompt Hydra to incorporate the configuration file with this name. In [Getting Started Part I](www.github.com/enlite-ai/maze/blob/main/tutorials/notebooks/getting_started_1.ipynb) we specified our algorithm like this (`...algorithm="ppo"...`).
* As configuration dictionary. We haven't encountered this so far and won't cover this in this tutorial.

On top of that, component properties can be overriden using the `overrides` argument. This is useful to change only a particular property without having to redefining the entire component, e.g. when changing a certain property in an algorithm configuration.

### Examples

Each algorithm in Maze (e.g. A2C, PPO, ...) has a corresponding configuration class - `A2CAlgorithmConfig`, `PPOAlgorithmConfiguration`, etc. It is common to set these algorithm hyperparameters based on the outcome of some optimization procedure. How we can change these values in Maze?

If you remember part I, the easiest way to specify an algorithm is to pass its name as string, e.g.

In [2]:
import gym
from maze.api.run_context import RunContext
from maze.core.wrappers.maze_gym_env_wrapper import GymMazeEnv

rc = RunContext(
    algorithm="a2c",
    env=lambda: GymMazeEnv(env=gym.make("CartPole-v0"))
)

INFO: Setting MKL_THREADING_LAYER=GNU to avoid PyTorch issues with conda!
INFO: Setting OMP_NUM_THREADS=1 to avoid performance drop when using distributed environments!


What if we want to change the learning rate? One possibility is to create an instance of the component, in this case `A2CAlgorithmConfig`, with the right parameters and pass it to `RunContext`:

In [3]:
from maze.train.trainers.a2c.a2c_algorithm_config import A2CAlgorithmConfig
from maze.train.parallelization.vector_env.sequential_vector_env import SequentialVectorEnv
from maze.train.trainers.common.evaluators.rollout_evaluator import RolloutEvaluator

alg_config = A2CAlgorithmConfig(
    n_epochs=1,
    epoch_length=25,
    patience=15,
    critic_burn_in_epochs=0,
    n_rollout_steps=100,
    lr=0.0005,
    gamma=0.98,
    gae_lambda=1.0,
    policy_loss_coef=1.0,
    value_loss_coef=0.5,
    entropy_coef=0.00025,
    max_grad_norm=0.0,
    device='cpu',
    rollout_evaluator=RolloutEvaluator(
        eval_env=SequentialVectorEnv([lambda: GymMazeEnv("CartPole-v0")]),
        n_episodes=1,
        model_selection=None,
        deterministic=True
    )
)

rc = RunContext(
    algorithm=alg_config,
    env=lambda: GymMazeEnv(env=gym.make("CartPole-v0"))
)

This however requires to create a new instance and specify all initialization arguments, which is not of interest to us here. Instead, we can use `overrides` to change only the property we're interested in:

In [4]:
rc = RunContext(
    algorithm="a2c",
    env=lambda: GymMazeEnv(env=gym.make("CartPole-v0")),
    overrides={"algorithm.lr": 0.00025}
)

This works for all components exposed in `RunContext`. Feel free to check out the documentation on [RunContext](https://maze-rl.readthedocs.io/en/latest/concepts_and_structure/run_context_overview.html) and [training](https://maze-rl.readthedocs.io/en/latest/workflow/training.html) to see more examples. Maze makes it fast and easy to adjust the configuration to suit your needs.

An example: Want to speed up your training? In Maze, [Runners](https://maze-rl.readthedocs.io/en/latest/scaling_training/runner_concept.html) are responsible for launching and managing individual runs. Without having to know much else about your runner you can simply change its `concurrency` parameter:

In [5]:
rc = RunContext(
    algorithm="a2c",
    env=lambda: GymMazeEnv(env=gym.make("CartPole-v0")),
    overrides={"runner.concurrency": 2}
)

## Wrappers

We refer to "wrappers" in the sense of environment wrappers as used in  [OpenAI's Gym](https://gym.openai.com/) (see e.g. [this](https://alexandervandekleut.github.io/gym-wrappers/) as a nice introduction). In a larger context a wrapper is a software design pattern that describes a class `B` wrapping a class `A`'s methods in order to use the functionality of `A` with `B`'s interface without modifying `A`'s code.

In Maze (and Gym), environment wrappers do not introduce interface changes. They add or modify functionality of the underlying entity. One example for this relates to observation preprocessing, e.g. normalization: This is functionality useful for many environments. A possibility to avoid a recurring implementation in every environment is to encapsulate this in a wrapper<sup>*</sup> preprocessing/normalizing observations after each `step()` and use only wrapped environments.

Since wrappers can wrap other wrappers, this underlying entity can be the environment itself or another wrapper. Calls to methods of the wrapped entity are propagated to the next layer, even if they are not explicitly implemented in the wrapper. E.g.: Wrapper class `ExampleWrapper` wraps environment class `ExampleEnv`. It only implements `step()` (by e.g. preprocessing input for `ExampleEnv.step()`). One could still invoke `reset()` on this wrapper instance though - the call is forwarded throughout the wrapper stack to the first class implementing this method (in this case `ExampleEnv`).

<sup>*</sup> Arguably this could also be solved via inheritance. However, doing so comes at the cost of a more complex (multi-)inheritance tree. It also lacks the flexibility that is inherent to wrappers: With wrappers it is possible to choose which functionality to extend your environment by at runtime - this is not the case with inheritance.


One way to use wrappers is to use their `wrap()` method on an instantiated environment:

In [6]:
from maze.core.wrappers.time_limit_wrapper import TimeLimitWrapper

env = GymMazeEnv(env=gym.make("CartPole-v0"))
wrapped_env = TimeLimitWrapper.wrap(env, max_episode_steps=3)

Here we use a `TimeLimitWrapper` that limits the number of elapsed steps before setting the environment to done. Let's test it:

In [7]:
done = False
i = 0
wrapped_env.reset()

while not done:
    _, _, done, _ = wrapped_env.step(env.action_space.sample())
    i += 1

print("Total number of iterations: {i}".format(i=i))

Total number of iterations: 3


As you can see, the total number of steps never exceeds 3 due to the wrapper's functionality. There are many applications for wrappers - observation normalization, limiting the max. number of steps to take, normalizing and clipping rewards etc.

It is very common in reinforcement learning to configure wrappers stacks facilitating the agent's learning process considering the nature of the learning problem and environment. For this reason Maze provides preconfigured wrapper stacks. E.g.: `pixel_obs` is suitable for the common problem setting of learning in the pixel space, e.g. for Atari games; `vector_obs` is configured for learning in the vectorized feature space, e.g. `CartPole`.

To illustrate we will train on the [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/) environment once without the `vector_obs` wrapper stack and once with and compare the results.

In [8]:
from maze.utils.notebooks import rollout
from maze.api.utils import working_directory


def train_and_eval_lunar_lander(use_vector_obs_wrappers: bool) -> float:
    """
    Trains and evaluates a RunContext on LunarLander-v2.
    :param use_vector_obs_wrappers: Whether to use vector_obs wrapper stack.
    :return: Mean return over 5 episodes.
    """

    wrappers = {"wrappers": "vector_obs"} if use_vector_obs_wrappers else {}
    runcon = RunContext(
        algorithm="ppo",
        env=lambda: GymMazeEnv(env=gym.make("LunarLander-v2")),
        silent=True,
        **wrappers
    )
    runcon.train(n_epochs=2)

    with working_directory(runcon.run_dir):
        mean_return = sum([
            rollout(runcon.env_factory(), runcon, 2500, log_dir=runcon.run_dir)
            for _ in range(10)
        ]) / 5

    return mean_return

print(
    """Mean returns:
        - {mr_wo} without vector_obs wrapper stack
        - {mr_w} with vector_obs wrapper stack.
    """.format(
        mr_wo=train_and_eval_lunar_lander(False),
        mr_w=train_and_eval_lunar_lander(True)
    )
)

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:08<00:00,  2.98it/s]


Mean returns:
        - -6.582475076712461 without vector_obs wrapper stack
        - 33.438312229389375 with vector_obs wrapper stack.
    


After training for two epochs, the mean return per episode is considerably higher with the `vector_obs` wrapper stack.


## Summary

This notebook...
* ...demonstrates how to specify and pass components to `RunContext` in different ways.
* ...shows how to set components' properties without having to specify the entire component by using `overrides`.
* ...describes Maze' configuration system from a high-level perspective.
* ...gives an introduction into the concept of environment wrappers and presents an example of how to apply a pre-configured wrapper stack.

### What's next?

* We recommend continuing with the [third part of the getting started series](www.github.com/enlite-ai/maze/blob/main/tutorials/notebooks/getting_started_3.ipynb).
* If you would like to see more notebooks covering other areas of Maze, feel free to [kick of a discussion on Github](https://github.com/enlite-ai/maze/discussions).